# Combination Generator

In [182]:
import pandas as pd
import numpy as np
import win32com.client as win32
lusas = win32.gencache.EnsureDispatch("Lusas.Modeller.21.1")
db = lusas.database()

Read the excel definitions

In [183]:
file_name = "12 Combinations.xlsx"
actions_sheet = "Actions"
do_print = False

In [184]:
df_permanent = pd.read_excel(file_name, sheet_name=actions_sheet, usecols=range(0,3), header=1).dropna()
if do_print : print(df_permanent.head())

In [185]:
df_variable = pd.read_excel(file_name, sheet_name=actions_sheet, usecols=range(4,9), header=1).dropna()
if do_print : print(df_variable.head())

In [186]:
df_prestress = pd.read_excel(file_name, sheet_name=actions_sheet, usecols=range(10,13), header=1).dropna()
if do_print : print(df_prestress.head())

In [187]:
df_accidental = pd.read_excel(file_name, sheet_name=actions_sheet, usecols=range(14,16), header=1).dropna()
if do_print : print(df_accidental.head())

In [ ]:
df_perm_loadcases = pd.read_excel(file_name, sheet_name="PermanentLoadcases")
print(df_perm_loadcases.head())
dict_perm_loadcases = df_perm_loadcases.set_index('Action Name').T.to_dict('list')
print(dict_perm_loadcases)

In [189]:
df_var_loadcases = pd.read_excel(file_name, sheet_name="VariableLoadcases")
if do_print : print(df_var_loadcases.head())
dict_var_loadcases = df_var_loadcases.set_index('Action Name').T.to_dict('list')

In [190]:
df_acc_loadcases = pd.read_excel(file_name, sheet_name="AccidentalLoadcases")
if do_print : print(df_acc_loadcases.head())

In [ ]:
def add_loadset_to_smart_comb(smart_comb, loadcase:str, beneficial:float, adverse:float) -> bool:
    if db.existsLoadset(loadcase):
        loadset = db.getLoadset(loadcase)
        type_code = loadset.getTypeCode()

        if type_code <= 2:
            smart_comb.addEntry(beneficial, (adverse-beneficial), loadset)
        elif type_code == 3:
            envelope1 = win32.CastTo(loadset, "IFEnvelope")
            envelope2 = envelope1.getAssocLoadset()
            smart_comb.addEntry(beneficial, (adverse-beneficial), envelope1)
            smart_comb.addEntry(beneficial, (adverse-beneficial), envelope2)
        elif type_code == 6:
            smart1 = win32.CastTo(loadset, "IFSmartCombination")
            smart2 = smart1.getAssocLoadset()
            smart_comb.addEntry(beneficial, (adverse-beneficial), smart1)
            smart_comb.addEntry(beneficial, (adverse-beneficial), smart2)
        return True
    else:
        return False


Equ 6.10

In [ ]:
uls_envelope = db.createEnvelope("ULS 6.10")

# Consider each variable action as a leading action
for i, row in df_variable.iterrows():
    action = row["Var Action Name"]
    load_factor = row["DesignFactor"]
    comb_factor = row["CombinationFactor"]
    
    smart_comb = db.createCombinationSmart(f"ULS - {action}")

    # Permanent actions
    for action_type in dict_perm_loadcases.keys():
        # Factors for this type
        beneficial = df_permanent[df_permanent['Perm Action Name'] == action_type]['Beneficial']
        adverse    = df_permanent[df_permanent['Perm Action Name'] == action_type]['Adverse']

        # LUSAS Loadcases
        for loadcase in dict_perm_loadcases[action_type]:
            if not isinstance(loadcase, str) and np.isnan(loadcase) : continue

            if not add_loadset_to_smart_comb(smart_comb, loadcase, beneficial, adverse):
                print(f"Permanent loadcase '{loadcase}' does not exists")


    # Variable actions
    for action_type in dict_var_loadcases.keys():
        # Determine variable factor
        if action_type == action:
            adverse = load_factor                        # Leading Variable
        else:
            adverse = load_factor * comb_factor          # Accompanying Variable

        # LUSAS Loadcases
        for loadcase in dict_var_loadcases[action_type]:
            if not isinstance(loadcase, str) and np.isnan(loadcase) : continue

            if not add_loadset_to_smart_comb(smart_comb, loadcase, 0.0, adverse):
                print(f"Variable loadcase '{loadcase}' does not exists")


    # Prestress actions
    for ip, rowp in df_prestress.iterrows():
        loadcase = rowp['Loadcase Name']
        if not isinstance(loadcase, str) and np.isnan(loadcase) : continue        
        
        # Factors for this type
        beneficial = rowp['Beneficial']
        adverse    = rowp['Adverse']

        if not add_loadset_to_smart_comb(smart_comb, loadcase, beneficial, adverse):
            print(f"Prestress loadcase '{loadcase}' does not exists")

    uls_envelope.addEntry(smart_comb)
    uls_envelope.addEntry(smart_comb.getAssocLoadset())
    smart_comb.setTreeParent(uls_envelope)
